# FNOL (First Notice of Loss) Processing

## Overview

This notebook processes First Notice of Loss (FNOL) data for insurance claims. It extracts, validates, and structures FNOL data with completeness checks to prepare it for downstream processing by fraud detection and policy verification teams.

## Workflow

1. **Import Dependencies** - Load required libraries (Strands, AWS Bedrock, JSON)
2. **AWS Configuration** - Configure Bedrock client and model
3. **Load FNOL Data** - Read and parse FNOL.json file
4. **FNOL Processing Agent** - Extract, validate, and structure claim data
5. **Data Quality Checks** - Flag missing fields and inconsistencies
6. **Output Structured Data** - Generate validated JSON for next stages


## 1. Install Dependencies

In [ ]:
%pip install -q -r requirements.txt --no-cache-dir --force-reinstall

In [ ]:
import IPython

IPython.Application.instance().kernel.do_shutdown(True)

In [1]:
!pip freeze | grep boto
!pip freeze | grep agentcore

aioboto3 @ file:///home/conda/feedstock_root/build_artifacts/aioboto3_1742196379442/work
aiobotocore @ file:///home/conda/feedstock_root/build_artifacts/aiobotocore_1741606508148/work
boto3==1.42.49
botocore==1.42.49
opentelemetry-instrumentation-boto==0.60b1
bedrock-agentcore==1.3.0
bedrock-agentcore-starter-toolkit==0.2.10


In [2]:
# Import libraries
import os
import json
import requests
import boto3
import time
from boto3.session import Session
from strands.tools import tool

# Get boto session
boto_session = Session()

In [3]:
![ ! -d "agents" ] && mkdir agents

## Create FNOL Processing Agent

This agent acts as a Claims Data Extraction Specialist that:
- Extracts key data points from FNOL documents
- Validates data completeness and consistency
- Flags potential data quality issues
- Generates structured JSON output for downstream processing

In [4]:
%%writefile agents/fnol-agent.py

import os
import logging
import asyncio
from mcp import stdio_client, StdioServerParameters
from strands import Agent, tool
from strands.models import BedrockModel
from strands.tools import tool
from strands.multiagent.a2a import A2AServer
from strands.tools.mcp import MCPClient
import argparse
from fastapi import FastAPI
import uvicorn

# Standard library imports
import json
from datetime import datetime

# AWS SDK
import boto3

from typing import Dict, Any, List
from dataclasses import dataclass



print("✓ All dependencies imported successfully")

########
# Configure the root strands logger
# logging.getLogger("strands").setLevel(logging.DEBUG)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Add a handler to see the logs
logging.basicConfig(
    format="%(levelname)s | %(name)s | %(message)s", 
    handlers=[logging.StreamHandler()]
)

# Setup tracing - commented out for now as this adds a lot of trace output that really isn't interesting
# NOTE: To send the OTEL data to an ADOT collector, additional exporter needs to be used
StrandsTelemetry().setup_console_exporter()

# AWS Configuration
AWS_REGION = "us-east-1"
# MODEL_ID = "anthropic.claude-sonnet-4-5-20250929-v1:0"

# Initialize Bedrock client to verify connectivity
bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=AWS_REGION
)



########


logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)



print(f"✓ AWS Bedrock configured for region: {AWS_REGION}")
print(f"✓ Using model: {MODEL_ID}")



# Initialize Bedrock model
model = BedrockModel(
    model_id="us.anthropic.claude-sonnet-4-20250514-v1:0",
    region_name="us-east-1"
)



# Create FNOL Processing Agent
fnol_agent = Agent(
    name="FNOL Data Extraction Specialist",
    description="Intelligent agent for automated auto insurance claims FNOL processing",
    model=model,
    system_prompt="""You are a Claims Data Extraction Specialist.

ROLE: Process First Notice of Loss (FNOL) forms and extract structured claim data.

INPUTS: Raw FNOL document/form data
OUTPUTS: Structured JSON with validated claim information

INSTRUCTIONS:
1. Extract key data points:
   - Policy number, claim date, incident location, claimant details
   - Incident description, damages reported, witnesses
   - Supporting documentation references

2. Validate data completeness:
   - Flag missing required fields
   - Identify inconsistencies in dates/locations
   - Check format compliance (policy numbers, contact info)

3. Flag potential data quality issues with specific error codes
4. If critical information is missing, generate specific follow-up questions

ERROR HANDLING: If form is illegible or severely incomplete, flag for manual review with detailed reasoning.

QUALITY CHECK: Ensure all extracted monetary amounts, dates, and identifiers are properly formatted.

OUTPUT FORMAT: 
Return the same structured JSON object provided in the input

Return, in a separete section, the list recommendations: Next steps or follow-up actions needed
"""
)

print("FNOL Processing Agent created")



################# A2A ################
app = FastAPI()
runtime_url = os.environ.get('AGENTCORE_RUNTIME_URL', 'http://127.0.0.1:9000/')
host, port = "0.0.0.0", 9000

a2a_server = A2AServer(
    agent=fnol_agent,
    http_url=runtime_url,
    serve_at_root=True,
    
)

@app.get("/ping")
def ping():
    return {"status": "healthy"}

# @app.on_event("startup")
# async def startup_event():
#     """Initialize MCP client on startup"""
#     await setup_agent_tools()

app.mount("/", a2a_server.to_fastapi_app())



if __name__ == "__main__":
    uvicorn.run(app, host=host, port=port)

################# A2A ################

Writing agents/fnol-agent.py


In [5]:
%%writefile agents/requirements.txt
boto3==1.40.50
bedrock-agentcore==0.1.7
strands-agents[a2a]
strands-agents-tools
pyyaml
ddgs

Overwriting agents/requirements.txt


#### Setup Cognito User Pool

In [6]:
from helpers.utils import setup_cognito_user_pool, reauthenticate_user

print("Setting up Amazon Cognito user pool...")
cognito_config = (
    setup_cognito_user_pool()
)  # You'll get your bearer token from this output cell.
print("Cognito setup completed ✓")

Setting up Amazon Cognito user pool...
Pool id: us-east-1_whucry5CD
Discovery URL: https://cognito-idp.us-east-1.amazonaws.com/us-east-1_whucry5CD/.well-known/openid-configuration
Client ID: 59bdr511f5ifrfoc2qe207vk3k
Bearer Token: eyJraWQiOiJ1VlJ0dnNoOVRqRWxSWkxFV1FjN0tGXC92a2R0dVpVdDBtMkFPNE02dHVFcz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiI5NDI4MTRiOC0yMDQxLTcwN2EtMjMwMC0zZWVmNGRiODhkNTYiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV93aHVjcnk1Q0QiLCJjbGllbnRfaWQiOiI1OWJkcjUxMWY1aWZyZm9jMnFlMjA3dmszayIsIm9yaWdpbl9qdGkiOiJkN2U3NDY3ZC1mZGE0LTRmM2YtYTJhYy0wZDYzYjIwMGFlODUiLCJldmVudF9pZCI6IjEwYmQ0ZDIzLTEwOWUtNDRkMi05ODIxLTVlM2Q5NmRiZTUyOCIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NzExOTQxNDUsImV4cCI6MTc3MTE5Nzc0NSwiaWF0IjoxNzcxMTk0MTQ1LCJqdGkiOiJiNDkwYjY3MS1iMmY1LTQ5NzEtYjJjNS03OTBmNGEzZGY1OGYiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.ScNgLhCSOb_uwq14EJaj2nOlFwl5Bjj16uzdKas-OgTr88EOE59Ti26ymU6W-guAzsEB23wWZCu

#### Create IAM Role for the Agents

In [7]:
from helpers.utils import create_agentcore_runtime_execution_role, AWS_FNOL_ROLE_NAME

execution_role_arn_fnol = create_agentcore_runtime_execution_role(AWS_FNOL_ROLE_NAME)

ℹ️ Role AWSFNOLAssistantBedrockAgentCoreRole-us-east-1 already exists
Role ARN: arn:aws:iam::161615149547:role/AWSFNOLAssistantBedrockAgentCoreRole-us-east-1


In [8]:
cognito_config.get("client_id")

'59bdr511f5ifrfoc2qe207vk3k'

In [9]:
cognito_config.get("discovery_url")

'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_whucry5CD/.well-known/openid-configuration'

In [10]:
from bedrock_agentcore_starter_toolkit import Runtime

agentcore_runtime_fnol_agent = Runtime()
fnol_agent_name="fnol_assistant"

region = boto_session.region_name

# Configure the deployment
response_fnol_agent = agentcore_runtime_fnol_agent.configure(
    entrypoint="agents/fnol-agent.py",
    execution_role=execution_role_arn_fnol,
    auto_create_ecr=True,
    requirements_file="agents/requirements.txt",
    region=region,
    agent_name=fnol_agent_name,
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [cognito_config.get("client_id")],
            "discoveryUrl": cognito_config.get("discovery_url"),
        }
    },
    protocol="A2A",
)

print("Configuration completed:", response_fnol_agent)

Entrypoint parsed: file=/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/agents/fnol-agent.py, bedrock_agentcore_name=fnol-agent
Memory disabled - agent will be stateless
Configuring BedrockAgentCore agent: fnol_assistant


💡 No container engine found (Docker/Finch/Podman not installed)

✓ Default deployment uses CodeBuild (no container engine needed), For local builds, install Docker, Finch, or 
Podman

Memory disabled
Found existing memory ID from previous launch: fnol_assistant_mem-iRbWou4Ycz
Network mode: PUBLIC


⚠️ Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64', so local builds
won't work.
Please use default launch command which will do a remote cross-platform build using code build.For deployment other
options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

📄 Using existing Dockerfile: 
/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/Dockerfile

Generated .dockerignore: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore
Changing default agent from 'aws_claimvalidation_assistant' to 'fnol_assistant'
Bedrock AgentCore configured: /home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/Multi-Agent-Collaboration/hackathon_ClaimsAdjudication/.dockerignore') runtime='None' runtime_type=None region='us-east-1' account_id='161615149547' execution_role='arn:aws:iam::161615149547:role/AWSFNOLAssistantBedrockAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True s3_path=None auto_create_s3=False memory_id=None network_mode='PUBLIC' network_subnets=None network_security_groups=None network_vpc_id=None


In [11]:
launch_result_fnol= agentcore_runtime_fnol_agent.launch()
print("Launch completed:", launch_result_fnol.agent_arn)

fnol_agent_arn = launch_result_fnol.agent_arn

🚀 Launching Bedrock AgentCore (cloud mode - RECOMMENDED)...
   • Deploy Python code directly to runtime
   • No Docker required (DEFAULT behavior)
   • Production-ready deployment

💡 Deployment options:
   • runtime.launch()                → Cloud (current)
   • runtime.launch(local=True)      → Local development
Memory disabled - skipping memory creation
Starting CodeBuild ARM64 deployment for agent 'fnol_assistant' to account 161615149547 (us-east-1)
Generated image tag: 20260215-222227-067
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: fnol_assistant
ECR repository available: 161615149547.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-fnol_assistant
Using execution role from config: arn:aws:iam::161615149547:role/AWSFNOLAssistantBedrockAgentCoreRole-us-east-1
Preparing CodeBuild project and uploading source...
Getting or creating CodeBuild execution role for agent: fnol_assistant
Role name: AmazonBedrockAgentCoreSDKCode

✅ Reusing existing ECR repository: 161615149547.dkr.ecr.us-east-1.amazonaws.com/bedrock-agentcore-fnol_assistant


Reusing existing CodeBuild execution role: arn:aws:iam::161615149547:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-93ed08f116
Using dockerignore.template with 47 patterns for zip filtering
Uploaded source to S3: fnol_assistant/source.zip
Updated CodeBuild project: bedrock-agentcore-fnol_assistant-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 8.3s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLOAD_SOURCE completed in 2.1s
🔄 BUILD started (total: 11s)
✅ BUILD completed in 17.5s
🔄 POST_BUILD started (total: 29s)
✅ POST_BUILD completed in 12.4s
🔄 COMPLETED started (total: 41s)
✅ COMPLETED completed in 1.0s
🎉 CodeBuild completed successfully in 0m 42s
CodeBuild completed successfully
CodeBuild project configuration saved
Deploying to Bedrock AgentCore...
Agent created/updated: arn:aws:bedrock-agentcore:us-east-1:16161

Launch completed: arn:aws:bedrock-agentcore:us-east-1:161615149547:runtime/fnol_assistant-MRDFkp2mkG


In [12]:
status_response = agentcore_runtime_fnol_agent.status()
status = status_response.endpoint["status"]

print(f"Final status: {status}")

Retrieved Bedrock AgentCore status for: fnol_assistant


Final status: READY


### Export and save outputs
Export variables to be used in next notebooks:

In [13]:
FNOL_AGENT_ID = launch_result_fnol.agent_id
FNOL_AGENT_ARN = launch_result_fnol.agent_arn
FNOL_AGENT_NAME = fnol_agent_name


FNOL_COGNITO_CLIENT_ID = cognito_config.get("client_id")
FNOL_COGNITO_SECRET = cognito_config.get("client_secret")
FNOL_DISCOVERY_URL = cognito_config.get("discovery_url")

%store FNOL_AGENT_ID
%store FNOL_AGENT_ARN
%store FNOL_AGENT_NAME
%store FNOL_COGNITO_CLIENT_ID
%store FNOL_COGNITO_SECRET
%store FNOL_DISCOVERY_URL

Stored 'FNOL_AGENT_ID' (str)
Stored 'FNOL_AGENT_ARN' (str)
Stored 'FNOL_AGENT_NAME' (str)
Stored 'FNOL_COGNITO_CLIENT_ID' (str)
Stored 'FNOL_COGNITO_SECRET' (str)
Stored 'FNOL_DISCOVERY_URL' (str)


In [14]:
FNOL_COGNITO_CLIENT_ID

'59bdr511f5ifrfoc2qe207vk3k'

In [15]:
cognito_config.get("client_id")

'59bdr511f5ifrfoc2qe207vk3k'

In [16]:
cognito_config.get("discovery_url")

'https://cognito-idp.us-east-1.amazonaws.com/us-east-1_whucry5CD/.well-known/openid-configuration'

In [17]:
from helpers.utils import put_ssm_parameter, SSM_FNOL_AGENT_ARN

put_ssm_parameter(SSM_FNOL_AGENT_ARN, FNOL_AGENT_ARN)


In [18]:
bearer_token = reauthenticate_user(
    cognito_config.get("client_id"), 
    cognito_config.get("client_secret")
)

In [19]:
bearer_token

'eyJraWQiOiJ1VlJ0dnNoOVRqRWxSWkxFV1FjN0tGXC92a2R0dVpVdDBtMkFPNE02dHVFcz0iLCJhbGciOiJSUzI1NiJ9.eyJzdWIiOiI5NDI4MTRiOC0yMDQxLTcwN2EtMjMwMC0zZWVmNGRiODhkNTYiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV93aHVjcnk1Q0QiLCJjbGllbnRfaWQiOiI1OWJkcjUxMWY1aWZyZm9jMnFlMjA3dmszayIsIm9yaWdpbl9qdGkiOiJmYWI2NThlYS04MGI3LTQ4YmQtYjAyNi0xN2NmNDk4OGVhYmUiLCJldmVudF9pZCI6ImY1ZDM2N2Y0LTEzN2UtNDBlNi1hMzhlLWYyYTcwYTBhZWM4ZSIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NzExOTQxOTMsImV4cCI6MTc3MTE5Nzc5MywiaWF0IjoxNzcxMTk0MTkzLCJqdGkiOiJjMjJiMmUyYy1hOGEwLTQ0YWItOGVlMi1iYjI1OTJiZjZmYmYiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.ERclK9wI4itpdj-i3OjmgJrFQ549m3y7kc7RIT_N7y6zsaFSUxCQVIDE1D7hC1ARGiIGUuRkOv5MsvTVCcxELcvNafauSoZFgyTYtYtWsdQqAS72UQAdrxUVrpty8Q4cYqtBR96r3TFLiEiFL28QpxqSQ3uURwxPfrwYZJAgioxEgEfXGLRcIoayjOTRGAp8gQA8bUgKxZd0nBq1FXTsu6S9vFl5VEK3UORiWNZsW5-_-TtLh4xB_QjR4KjKXq9VUSHBHWlhuceltc4VsepE5rDJ_9ABTF0XL4OhLj64R2PP92LvUv

In [20]:
import logging
from uuid import uuid4
from urllib.parse import quote

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

def fetch_agent_card(agent_arn):
    # URL encode the agent ARN
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/.well-known/agent-card.json"
    logger.info(url)
    # Generate a unique session ID
    session_id = str(uuid4())
    logger.info(f"Generated session ID: {session_id}")

    # Set headers
    headers = {
        'Accept': '*/*',
        'Authorization': f'Bearer {bearer_token}',
        'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id,
        'X-Amzn-Trace-Id': f'aws_docs_assistant_{session_id}'
    }

    try:
        # Make the request
        response = requests.get(url, headers=headers)
        response.raise_for_status()

        # Parse and pretty print JSON
        agent_card = response.json()
        logger.info(json.dumps(agent_card, indent=2))

        return agent_card

    except requests.exceptions.RequestException as e:
        logger.error(f"Error fetching agent card: {e}")
        return None

In [22]:
fetch_agent_card(fnol_agent_arn)

{'capabilities': {'streaming': True},
 'defaultInputModes': ['text'],
 'defaultOutputModes': ['text'],
 'description': 'A Single agent with Claims Validation tools capabilities',
 'name': 'Claims Validator - ARC',
 'preferredTransport': 'JSONRPC',
 'protocolVersion': '0.3.0',
 'skills': [{'description': 'Retrieves knowledge based on the provided text from Amazon Bedrock Knowledge Bases.\n\nKey Features:\n1. Semantic Search:\n   - Vector-based similarity matching\n   - Relevance scoring (0.0-1.0)\n   - Score-based filtering\n\n2. Advanced Configuration:\n   - Custom result limits\n   - Score thresholds\n   - Regional support\n   - Multiple knowledge bases\n\n3. Response Format:\n   - Sorted by relevance\n   - Includes metadata\n   - Source tracking\n   - Score visibility\n\n4. Example Response:\n   {\n     "content": {\n       "text": "Document content...",\n       "type": "TEXT"\n     },\n     "location": {\n       "customDocumentLocation": {\n         "id": "document_id"\n       },\n 

### Test agents
Now, let's invoke the first agent, using A2A:

In [23]:
import asyncio
import logging
import os
from uuid import uuid4

import httpx
from a2a.client import A2ACardResolver, ClientConfig, ClientFactory
from a2a.types import Message, Part, Role, TextPart

logging.basicConfig(level=logging.ERROR)
logger = logging.getLogger(__name__)

DEFAULT_TIMEOUT = 300  # set request timeout to 5 minutes

def format_agent_response(response):
    """Extract and format agent response for human readability."""
    # Get the main response text from artifacts
    if response.artifacts and len(response.artifacts) > 0:
        artifact = response.artifacts[0]
        if artifact.parts and len(artifact.parts) > 0:
            return artifact.parts[0].root.text
    
    # Fallback: concatenate all agent messages from history
    agent_messages = [
        msg.parts[0].root.text 
        for msg in response.history 
        if msg.role.value == 'agent' and msg.parts
    ]
    return ''.join(agent_messages)


def create_message(*, role: Role = Role.user, text: str) -> Message:
    return Message(
        kind="message",
        role=role,
        parts=[Part(TextPart(kind="text", text=text))],
        message_id=uuid4().hex,
    )

async def send_sync_message(agent_arn, message: str):
    # Get runtime URL from environment variable
    escaped_agent_arn = quote(agent_arn, safe='')

    # Construct the URL
    runtime_url = f"https://bedrock-agentcore.{region}.amazonaws.com/runtimes/{escaped_agent_arn}/invocations/"
    
    # Generate a unique session ID
    session_id = str(uuid4())
    print(f"Generated session ID: {session_id}")

    # Add authentication headers for AgentCore
    headers = {"Authorization": f"Bearer {bearer_token}",
              'X-Amzn-Bedrock-AgentCore-Runtime-Session-Id': session_id}
        
    async with httpx.AsyncClient(timeout=DEFAULT_TIMEOUT, headers=headers) as httpx_client:
        # Get agent card from the runtime URL
        resolver = A2ACardResolver(httpx_client=httpx_client, base_url=runtime_url)
        agent_card = await resolver.get_agent_card()
        print(agent_card)

        # Agent card contains the correct URL (same as runtime_url in this case)
        # No manual override needed - this is the path-based mounting pattern

        # Create client using factory
        config = ClientConfig(
            httpx_client=httpx_client,
            streaming=False,  # Use non-streaming mode for sync response
        )
        factory = ClientFactory(config)
        client = factory.create(agent_card)

        # Create and send message
        msg = create_message(text=message)

        # With streaming=False, this will yield exactly one result
        async for event in client.send_message(msg):
            if isinstance(event, Message):
                logger.info(event.model_dump_json(exclude_none=True, indent=2))
                return event
            elif isinstance(event, tuple) and len(event) == 2:
                # (Task, UpdateEvent) tuple
                task, update_event = event
                logger.info(f"Task: {task.model_dump_json(exclude_none=True, indent=2)}")
                if update_event:
                    logger.info(f"Update: {update_event.model_dump_json(exclude_none=True, indent=2)}")
                return task
            else:
                # Fallback for other response types
                logger.info(f"Response: {str(event)}")
                return event

## 7. Process FNOL Data

In [24]:
# Load FNOL JSON data
with open('data/FNOL.json', 'r') as f:
    fnol_data = json.load(f)

print("✓ FNOL data loaded successfully")
print(f"\nClaim Number: {fnol_data['fnol']['claimNumber']}")
print(f"Report Date: {fnol_data['fnol']['reportDate']}")
print(f"Policyholder: {fnol_data['policyholder']['firstName']} {fnol_data['policyholder']['lastName']}")
print(f"Vehicle: {fnol_data['vehicle']['insuredVehicle']['year']} {fnol_data['vehicle']['insuredVehicle']['make']} {fnol_data['vehicle']['insuredVehicle']['model']}")

✓ FNOL data loaded successfully

Claim Number: CL-2023-1156789
Report Date: 2023-11-21
Policyholder: John Smith
Vehicle: 2020 Honda Accord


In [25]:
# Create processing prompt
fnol_prompt = f"""
Please process the following FNOL (First Notice of Loss) data.

FNOL Data:
{json.dumps(fnol_data, indent=2)}
"""

print("=" * 80)
print("PROCESSING FNOL DATA")
print("=" * 80)
print("\n FNOL prompt is ready ...\n")

PROCESSING FNOL DATA

 FNOL prompt is ready ...



In [26]:
result = await send_sync_message(fnol_agent_arn, fnol_prompt)
formatted_output = format_agent_response(result)
print(formatted_output)

Generated session ID: 45aa7416-636f-4a2b-8bf7-52b969892418
additional_interfaces=None capabilities=AgentCapabilities(extensions=None, push_notifications=None, state_transition_history=None, streaming=True) default_input_modes=['text'] default_output_modes=['text'] description='A Single agent with Claims Validation tools capabilities' documentation_url=None icon_url=None name='Claims Validator - ARC' preferred_transport='JSONRPC' protocol_version='0.3.0' provider=None security=None security_schemes=None signatures=None skills=[AgentSkill(description='Retrieves knowledge based on the provided text from Amazon Bedrock Knowledge Bases.\n\nKey Features:\n1. Semantic Search:\n   - Vector-based similarity matching\n   - Relevance scoring (0.0-1.0)\n   - Score-based filtering\n\n2. Advanced Configuration:\n   - Custom result limits\n   - Score thresholds\n   - Regional support\n   - Multiple knowledge bases\n\n3. Response Format:\n   - Sorted by relevance\n   - Includes metadata\n   - Source t